### **Graph**

#### Google Drive 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 사용할 module import

In [2]:
import pandas as pd
import networkx as nx
import numpy as np

#### 데이터 불러오기
**# Columns Information**
- company: 회사명
- investment: 누적투자금액
- sales: 매출
- employee: 고용수
- category: 카테고리
- keyword: 키워드
- establishment: 설립년도
- view: 최근 조회수
- total_mean: 최종 순위

In [3]:
# CSV 파일 불러오기
nodes = pd.read_csv('/content/drive/MyDrive/2024-1 graph-mining/final_project/sort_top50.csv')

#### [Node] : startup name
- 회사명 : company
- 크기 : 매출액

In [4]:
# 노드 데이터프레임에서 노드 리스트 생성
nodes_list = nodes['company'].tolist()

#### [Edge] : category
- 같은 카테고리 속한 기업 연결
- 가중치 : 년도

In [5]:
# 엣지 생성을 위한 데이터프레임 생성
edges_df = nodes[['company', 'category']].copy()

#### 그래프 생성
- Node와 Edge 연결
- Node의 attribute 추가 -> 투자금액에 따라 노드 크기

In [6]:
# 노드 리스트를 딕셔너리 형태로 변환하여 노드 ID로 사용
nodes_dict = {node: idx for idx, node in enumerate(nodes_list)}

In [7]:
# 엣지를 연결할 그래프 객체 생성
G = nx.Graph()

In [9]:
# 노드 추가 및 속성 설정
for idx, row in nodes.iterrows():
    company = row['company']
    employee = row['employee']
    G.add_node(company, employee=employee)

In [10]:
# 엣지 추가
for _, row in edges_df.iterrows():
    company = row['company']
    categories = row['category'].split(',')  # 쉼표를 기준으로 카테고리를 분할
    for category in categories:
        # 같은 category에 속한 노드를 연결
        connected_nodes = edges_df[edges_df['category'].str.contains(category)]['company'].tolist()
        for connected_node in connected_nodes:
            if company != connected_node:  # 자기 자신과는 연결하지 않도록 조건 추가
                G.add_edge(company, connected_node)

#### Edge Weight 추가
- Vectorization column의 값을 활용하여 노드 간 Cosine Similarity 계산


In [11]:
# GraphML 파일로 저장하는 함수 정의
def save_graphml(graph, filename):
    nx.write_graphml_lxml(graph, filename)

#### 생성된 Node, Edge 개수 확인

In [12]:
# 노드 개수 출력
print("Number of nodes:", G.number_of_nodes())

# 엣지 개수 출력
print("Number of edges:", G.number_of_edges())

Number of nodes: 50
Number of edges: 308


#### GraphML 파일로 저장

In [13]:
# GraphML 파일로 저장
save_graphml(G, "startup_graph_3.graphml")